# Testing Transfer Learning
Transfer Learning is the use of a pre-trained convolutional neural network in a **new domain**. It can be used as:
- **Fixed Feature Extractor:** we use last flatten vector as feature extractor to be used in a SVM
- **Pre-trained CNN:** only the classifier is trained on the new dataset. This is recommended for small datasets. For instance if ImageNet has 1000 classes, we substitute the last CNN for one where there are as many classes as outputs our dataset has
- **Fine-Tunning:** Initial layers are frozen. The remainining layers are re-trained. It is recommended for Datasets of *Medium Size*

## Importing Libraries
Libraries are imported and model structure is displayed

In [4]:
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image

model = VGG16(include_top=True, weights='imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Model Last Layer

In [11]:
model.layers[-1].get_config()

{'name': 'predictions',
 'trainable': True,
 'units': 1000,
 'activation': 'softmax',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

## Reading Image
Image is preprocessed as indicated by VGG
There is a dimension processing

In [5]:
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = preprocess_input(x)
print('Input Image Shape: ', x.shape)

Input Image Shape:  (1, 224, 224, 3)


## Predictions:
We print top 3 predictions with their % of probability

In [12]:
preds = model.predict(x)
print('Predicted: ', decode_predictions(preds, top=3)[0])

Predicted:  [('n02504013', 'Indian_elephant', 0.71757305), ('n02504458', 'African_elephant', 0.24314512), ('n01871265', 'tusker', 0.036461875)]


## Feature Extraction
Now we have model VGG16 without the Classifier and the Softmax. It is the full model until the last block that ends in max pooling leaving an array of 512 

In [13]:
model_fe = VGG16(include_top=False, weights='imagenet')
model_fe.summary()
model_fe.layers[-1].get_config()
# x must be preprocessed as indicated above


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

{'name': 'block5_pool',
 'trainable': True,
 'pool_size': (2, 2),
 'padding': 'valid',
 'strides': (2, 2),
 'data_format': 'channels_last'}

Getting the Feature vector

In [15]:
feature_vector = model_fe.predict(x)
print(feature_vector.shape)


(1, 7, 7, 512)
